In [3]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [39]:
ventas = pd.DataFrame({
    'persona': ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'A'],
    'categoria': ['ropa', 'calzado', 'ropa', 'accesorios', 'calzado', 'ropa', 'accesorios', 'ropa'],
    'compra': [100, 200, 50, 75, 125, 300, 400, 50],
})
ventas.head(10)

,persona,categoria,compra,times
0,A,ropa,100,1
1,A,calzado,200,1
2,B,ropa,50,1
3,B,accesorios,75,1
4,B,calzado,125,1
5,C,ropa,300,1
6,C,accesorios,400,1
7,A,ropa,50,1


In [99]:
def proportion_categories(dataframe, pet, val1, val2, val3):
    '''
    Calcular las proporciones de productos comprados por cada usuario
    pet = 0 for categories, 1 for money
    val1: variable for calculating proportions (persona que compra)
    val2: variable to group by (categoria compra)
    val3: variable to group by (valor compra)
    '''

    if pet==0:

        # Calcula las proporciones y crea una nueva columna en el DataFrame final
        proporciones = dataframe.groupby(val1)[val2].value_counts(normalize=True).reset_index(name='proporcion')
        
        # Crear una tabla pivote
        proporciones = proporciones.pivot_table(
            index=val1,
            columns=val2,
            values='proporcion',
            fill_value=0
        ).reset_index()

    else:

        # Retirar las variables datetime64
        dataframe = dataframe.select_dtypes(exclude=['datetime64'])

        # Calcula la suma de dinero por usuario y categoría
        df_sum = dataframe.groupby([val1, val2]).sum().reset_index()

        # Calcula la suma total de dinero por usuario
        df_total = dataframe.groupby(val1)[val3].sum().reset_index()

        # Fusiona los DataFrames para obtener las proporciones
        proporciones = pd.merge(df_sum, df_total, on=val1, suffixes=('_categoria', '_total'))

        # Calcula las proporciones y crea una nueva columna en el DataFrame final
        proporciones['proporcion'] = proporciones[val3 + '_categoria'] / proporciones[val3 + '_total']

        # Crear una tabla pivote
        proporciones = proporciones.pivot_table(
            index=val1,
            columns=val2,
            values='proporcion',
            fill_value=0
        ).reset_index()

    return proporciones

In [101]:
salida = proportion_categories(ventas, 0, 'persona', 'categoria', 'compra')
salida.head()

categoria,persona,accesorios,calzado,ropa
0,A,0.000000,0.333333,0.666667
1,B,0.333333,0.333333,0.333333
2,C,0.500000,0.000000,0.500000


In [102]:
salida = proportion_categories(ventas, 1, 'persona', 'categoria', 'compra')
salida.head()

categoria,persona,accesorios,calzado,ropa
0,A,0.000000,0.571429,0.428571
1,B,0.300000,0.500000,0.200000
2,C,0.571429,0.000000,0.428571


In [ ]:
print('ok_')

ok_
